1 subject. Feature extraction. Classification joining all frequency bands.

In [254]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score    
from sklearn import svm
from sklearn.svm import LinearSVC


#def select_file(file):
    #Lectura mediante pandas
f = "data_preprocessed_python\\s01.dat"
subject_id = 1
dictRaw = pd.read_pickle(f)

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
    #Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

In [255]:
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 30),
                'Gamma': (30, 50)}

In [256]:
# FOURIER
# SE COGE EL PRIMER VÍDEOS, Df_VIDEOS[0]
def do_stft(video, channel):
    # Array 
    x = df_videos[video][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)

    return Zxx


In [257]:
#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]

    return eeg_band_fft


In [258]:
# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
def channel_freq(eeg_band_fft):
        
        freq = np.zeros((5, 64))        # 5 bandas, 64 posiciones de los datos

        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:

                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos al video 0, un canal concreto de un sujeto concreto

In [259]:
#df_file_videos = select_file('d\\s01.dat')
# Obtener la división por bandas de frecuencia de cada uno de los canales del video 0
for video in range(0, 40):
    delta_bands = []
    theta_bands = []
    beta_bands = []
    alpha_bands = []
    gamma_bands = []

    for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
        Zxx = do_stft(video, channel)
        eeg_band_fft = make_bands(Zxx)
        freq = channel_freq(eeg_band_fft)

        delta_bands.append(freq[0])
        theta_bands.append(freq[1])
        alpha_bands.append(freq[2])
        beta_bands.append(freq[3])
        gamma_bands.append(freq[4])
    # En freq tenemos la media de todas las bandas de frecuencia de cada uno de los canales del vídeo 0

In [260]:
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['D-FP1','D-AF3','D-F7','D-F3','D-FC1','D-FC5','D-T7','D-C3','D-CP1','D-CP5','D-P7','D-P3','D-Pz','D-PO3','D-O1','D-Oz','D-O2','D-PO4','D-P4','D-P8','D-CP6','D-CP2','D-C4','D-T8','D-FC6','D-FC2','D-F4','D-F8','D-AF4','D-Fp2','D-Fz','D-Cz']
delta_df

rows = list()
for _,row in delta_df.iterrows():
    rows += [row]*40
aux_delta = pd.DataFrame(rows).reset_index(drop=True)

In [261]:
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns =  ['T-FP1','T-AF3','T-F7','T-F3','T-FC1','T-FC5','T-T7','T-C3','T-CP1','T-CP5','T-P7','T-P3','T-Pz','T-PO3','T-O1','T-Oz','T-O2','T-PO4','T-P4','T-P8','T-CP6','T-CP2','T-C4','T-T8','T-FC6','T-FC2','T-F4','T-F8','T-AF4','T-Fp2','T-Fz','T-Cz']
theta_df

rows = list()
for _,row in theta_df.iterrows():
    rows += [row]*40
aux_theta = pd.DataFrame(rows).reset_index(drop=True)

In [262]:
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['A-FP1','A-AF3','A-F7','A-F3','A-FC1','A-FC5','A-T7','A-C3','A-CP1','A-CP5','A-P7','A-P3','A-Pz','A-PO3','A-O1','A-Oz','A-O2','A-PO4','A-P4','A-P8','A-CP6','A-CP2','A-C4','A-T8','A-FC6','A-FC2','A-F4','A-F8','A-AF4','A-Fp2','A-Fz','A-Cz']
alpha_df

rows = list()
for _,row in alpha_df.iterrows():
    rows += [row]*40
aux_alpha = pd.DataFrame(rows).reset_index(drop=True)

In [263]:
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['B-FP1','B-AF3','B-F7','B-F3','B-FC1','B-FC5','B-T7','B-C3','B-CP1','B-CP5','B-P7','B-P3','B-Pz','B-PO3','B-O1','B-Oz','B-O2','B-PO4','B-P4','B-P8','B-CP6','B-CP2','B-C4','B-T8','B-FC6','B-FC2','B-F4','B-F8','B-AF4','B-Fp2','B-Fz','B-Cz']
beta_df

rows = list()
for _,row in beta_df.iterrows():
    rows += [row]*40
aux_beta = pd.DataFrame(rows).reset_index(drop=True)

In [264]:
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['G-FP1','G-AF3','G-F7','G-F3','G-FC1','G-FC5','G-T7','G-C3','G-CP1','G-CP5','G-P7','G-P3','G-Pz','G-PO3','G-O1','G-Oz','G-O2','G-PO4','G-P4','G-P8','G-CP6','G-CP2','G-C4','G-T8','G-FC6','G-FC2','G-F4','G-F8','G-AF4','G-Fp2','G-Fz','G-Cz']
gamma_df

rows = list()
for _,row in gamma_df.iterrows():
    rows += [row]*40
aux_gamma = pd.DataFrame(rows).reset_index(drop=True)

In [265]:
df_labels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*64
aux_labels = pd.DataFrame(rows).reset_index(drop=True)

In [266]:
data_delta = aux_delta.merge(aux_labels, left_index=True, right_index=True)
data_theta = aux_theta.merge(aux_labels, left_index=True, right_index=True)
data_alpha = aux_alpha.merge(aux_labels, left_index=True, right_index=True)
data_beta = aux_beta.merge(aux_labels, left_index=True, right_index=True)
data_gamma = aux_gamma.merge(aux_labels, left_index=True, right_index=True)

# Todas las bandas una al lado de otra

In [267]:
frames = [aux_delta, aux_theta, aux_alpha, aux_beta, aux_gamma]
aux_all_bands = pd.concat(frames, axis=1)
all_bands_channels = ['D-FP1','D-AF3','D-F7','D-F3','D-FC1','D-FC5','D-T7','D-C3','D-CP1','D-CP5','D-P7','D-P3','D-Pz','D-PO3','D-O1','D-Oz','D-O2','D-PO4','D-P4','D-P8','D-CP6','D-CP2','D-C4','D-T8','D-FC6','D-FC2','D-F4','D-F8','D-AF4','D-Fp2','D-Fz','D-Cz', 'T-FP1','T-AF3','T-F7','T-F3','T-FC1','T-FC5','T-T7','T-C3','T-CP1','T-CP5','T-P7','T-P3','T-Pz','T-PO3','T-O1','T-Oz','T-O2','T-PO4','T-P4','T-P8','T-CP6','T-CP2','T-C4','T-T8','T-FC6','T-FC2','T-F4','T-F8','T-AF4','T-Fp2','T-Fz','T-Cz', 'A-FP1','A-AF3','A-F7','A-F3','A-FC1','A-FC5','A-T7','A-C3','A-CP1','A-CP5','A-P7','A-P3','A-Pz','A-PO3','A-O1','A-Oz','A-O2','A-PO4','A-P4','A-P8','A-CP6','A-CP2','A-C4','A-T8','A-FC6','A-FC2','A-F4','A-F8','A-AF4','A-Fp2','A-Fz','A-Cz', 'B-FP1','B-AF3','B-F7','B-F3','B-FC1','B-FC5','B-T7','B-C3','B-CP1','B-CP5','B-P7','B-P3','B-Pz','B-PO3','B-O1','B-Oz','B-O2','B-PO4','B-P4','B-P8','B-CP6','B-CP2','B-C4','B-T8','B-FC6','B-FC2','B-F4','B-F8','B-AF4','B-Fp2','B-Fz','B-Cz', 'G-FP1','G-AF3','G-F7','G-F3','G-FC1','G-FC5','G-T7','G-C3','G-CP1','G-CP5','G-P7','G-P3','G-Pz','G-PO3','G-O1','G-Oz','G-O2','G-PO4','G-P4','G-P8','G-CP6','G-CP2','G-C4','G-T8','G-FC6','G-FC2','G-F4','G-F8','G-AF4','G-Fp2','G-Fz','G-Cz']

In [268]:
all_bands_df = aux_all_bands.merge(aux_labels, left_index=True, right_index=True)

# Channel selection

Literature : https://sci-hub.do/https://link.springer.com/article/10.1007/s40708-017-0069-3

In [269]:
literature_selection = pd.DataFrame(data=all_bands_df, columns=['D-P3', 'T-P3', 'A-P3', 'B-P3', 'G-P3', 'D-FC2','T-FC2','A-FC2','B-FC2','G-FC2', 'D-AF3','T-AF3','A-AF3','B-AF3','G-AF3', 'D-O1','T-O1','A-O1','B-O1','G-O1', 'D-FP1','T-FP1','A-FP1','B-FP1','G-FP1', 'valence'])

In [270]:
data = literature_selection.copy()
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)


clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, np.ravel(yTrain))
predict = clf.predict(xTest)

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

rf_acc = accuracy_score(yTest, predict)
svm_acc = accuracy_score(yTest, svm_predict)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


Own

In [271]:
data = all_bands_df.copy()
data = data.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

In [272]:
clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, np.ravel(yTrain))

series_importance = pd.Series(abs(clf.feature_importances_), index=all_bands_channels).nlargest(25)
df_importances = series_importance.to_frame()
res = df_importances.index

own_selection = pd.DataFrame(data=all_bands_df, columns=res)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


In [273]:
svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))

svm_series_importance = pd.Series(abs(svm_clf.coef_[0]), index=all_bands_channels).nlargest(25)
df_svm_importances = svm_series_importance.to_frame()
svm_res = df_svm_importances.index

svm_own_selection = pd.DataFrame(data=all_bands_df, columns=svm_res)

C:\Python39\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [274]:
data = own_selection.merge(aux_labels, left_index=True, right_index=True)

data = data.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, np.ravel(yTrain))
predict = clf.predict(xTest)

rf_acc2 = accuracy_score(yTest, predict)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.0s finished


In [275]:
data = svm_own_selection.merge(aux_labels, left_index=True, right_index=True)
data = data.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] >5, 'happy', 'sad')

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)
    
svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

svm_acc2 = accuracy_score(yTest, svm_predict)

In [276]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy']

data_CSV = [subject_id, 4, "STFT", "All bands", "Literature", "Random Forest", rf_acc], [subject_id, 4, "STFT", "All bands", "Literature", "SVM", svm_acc], [subject_id, 4, "STFT", "All bands", "Own", "Random Forest", rf_acc2], [subject_id, 4, "STFT", "All bands", "Own", "SVM", svm_acc2]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment4.csv", mode="a", header=col_names, index=False)